In [1]:
import pandas as pd
from src.recsys_baseline import VSKNN_STAN
from src.recsys_rl import rl_recommender
from src.evaluation import evaluate_sessions, MRR, HitRate, StrategicHybrid

## load datasets

In [2]:
train = pd.read_csv(r'data/premade datasets/retailrocket/events_train_full.txt',
                    sep='\t')
display(train.head())
display(train.shape)

,Time,UserId,Type,ItemId,SessionId
0,1442004589,0,view,285930,1
1,1442004759,0,view,357564,1
2,1442004917,0,view,67045,1
3,1438969904,2,view,325215,3
4,1438970013,2,view,325215,3


(1184988, 5)

In [3]:
test = pd.read_csv(r'data/premade datasets/retailrocket/events_test.txt',
                    sep='\t')
display(test.head())
display(test.shape)

,Time,UserId,Type,ItemId,SessionId
0,1442535624,1840,view,129332,2301
1,1442535780,1840,view,358485,2301
2,1442535949,1840,view,287227,2301
3,1442536030,1840,view,290227,2301
4,1442458998,2937,view,81554,3742


(3936, 5)

## Hybrid model (reward: click=3, addtocart=8, purchase=10)

In [4]:
model_1 = rl_recommender(
    n_history=5,
    file_path=r'trained agents/retailrocket/click=3, addtocart=8, purchase=10/20240627_135343 - n_hist 5 state_size 5 action_size 466865.pth',
    state_size=5,
    action_size=466865,
    mode='predicting'
)

model_2 = VSKNN_STAN(k=500,
                     sample_size=1000,
                     similarity='cosine',
                    #    stan:  
                     lambda_spw=7.24,  
                     lambda_snh=100,  
                     lambda_inh=3.62,  
                    #    vsknn:  
                     lambda_ipw=3.62, 
                     lambda_idf=1
                       )

# Initialize the hybrid model
model = StrategicHybrid(algorithms=[model_1, model_2],
                        thresholds=[5]) # after 5 recommendations, the second algorithm is used
# Fit the hybrid model
model.fit(train)

Initializing offline training agent


/home/mees/anaconda3/envs/master_thesis/lib/python3.9/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Loading pretrained model from: trained agents/retailrocket/20240627_135343 - n_hist 5 state_size 5 action_size 466865.pth
Pretrained model successfully loaded


In [5]:
metrics = [MRR(20),
           HitRate(20)]

evaluate_sessions(pr=model,
                  metrics= metrics,
                  test_data=test,
                  train_data=train,
                  cut_off=20)

START evaluation of  3936  actions in  1125  sessions
    eval process:  0  of  3936  actions:  0.0  %
    eval process:  1000  of  3936  actions:  25.406504065040654  %
    eval process:  2000  of  3936  actions:  50.81300813008131  %
    eval process:  3000  of  3936  actions:  76.21951219512195  %
END evaluation


[('MRR@20: ',
  0.0509260510443804,
  'Bin: ;\nPrecision@20: ;',
  'Pos: ;0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;21;22;23;24;25;26;27;28;29;30;31;32;33;34;35;36;37;38;39;40;41;42;43;44;45;46;47;48;49;50;51;52;53;54;55;56;57;58;59;60;61;62;63;64;65;66;67;68;\nPrecision@20: ;0.0;0.0;0.0;0.0;0.0;0.23942099567099567;0.31159953880542113;0.2682329622132254;0.319805154407618;0.3472222222222222;0.245109201575367;0.2618351886209029;0.28474747474747475;0.2072916666666667;0.23703703703703705;0.26095238095238094;0.19166666666666668;0.1987179487179487;0.16666666666666666;0.061294765840220394;0.029513888888888888;0.15422077922077923;0.046703296703296704;0.12012987012987013;0.21428571428571427;0.07222222222222222;0.0;0.06666666666666667;0.32857142857142857;0.11333333333333333;0.1181818181818182;0.025;0.0;0.016666666666666666;0.041666666666666664;0.09375;0.0;0.037037037037037035;0.0;0.0434173669467787;0.037037037037037035;0.047619047619047616;0.041666666666666664;0.025641025641025644;0.3

### 10 interactions

In [6]:
model_1 = rl_recommender(
    n_history=10,
    file_path=r'trained agents/retailrocket/click=3, addtocart=8, purchase=10/20240627_141013 - n_hist 10 state_size 10 action_size 466865.pth',
    state_size=10,
    action_size=466865,
    mode='predicting'
)

model_2 = VSKNN_STAN(k=500,
                     sample_size=1000,
                     similarity='cosine',
                    #    stan:  
                     lambda_spw=7.24,  
                     lambda_snh=100,  
                     lambda_inh=3.62,  
                    #    vsknn:  
                     lambda_ipw=3.62, 
                     lambda_idf=1
                       )

# Initialize the hybrid model
model = StrategicHybrid(algorithms=[model_1, model_2],
                        thresholds=[10]) # after 10 recommendations, the second algorithm is used
# Fit the hybrid model
model.fit(train)

Initializing offline training agent
Loading pretrained model from: trained agents/retailrocket/20240627_141013 - n_hist 10 state_size 10 action_size 466865.pth
Pretrained model successfully loaded


In [7]:
metrics = [MRR(20),
           HitRate(20)]

evaluate_sessions(pr=model,
                  metrics= metrics,
                  test_data=test,
                  train_data=train,
                  cut_off=20)

START evaluation of  3936  actions in  1125  sessions
    eval process:  0  of  3936  actions:  0.0  %
    eval process:  1000  of  3936  actions:  25.406504065040654  %
    eval process:  2000  of  3936  actions:  50.81300813008131  %
    eval process:  3000  of  3936  actions:  76.21951219512195  %
END evaluation


[('MRR@20: ',
  0.020423638981978468,
  'Bin: ;\nPrecision@20: ;',
  'Pos: ;0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;21;22;23;24;25;26;27;28;29;30;31;32;33;34;35;36;37;38;39;40;41;42;43;44;45;46;47;48;49;50;51;52;53;54;55;56;57;58;59;60;61;62;63;64;65;66;67;68;\nPrecision@20: ;0.0004967320261437909;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.245109201575367;0.2618351886209029;0.28474747474747475;0.2072916666666667;0.23703703703703705;0.26095238095238094;0.19166666666666668;0.1987179487179487;0.16666666666666666;0.061294765840220394;0.029513888888888888;0.15422077922077923;0.046703296703296704;0.12012987012987013;0.21428571428571427;0.07222222222222222;0.0;0.06666666666666667;0.32857142857142857;0.11333333333333333;0.1181818181818182;0.025;0.0;0.016666666666666666;0.041666666666666664;0.09375;0.0;0.037037037037037035;0.0;0.0434173669467787;0.037037037037037035;0.047619047619047616;0.041666666666666664;0.025641025641025644;0.3703703703703704;0.0;0.020833333333333332;0.0;0.111111111

### 15 interactions

In [8]:
model_1 = rl_recommender(
    n_history=15,
    file_path=r'trained agents/retailrocket/click=3, addtocart=8, purchase=10/20240627_142646 - n_hist 15 state_size 15 action_size 466865.pth',
    state_size=15,
    action_size=466865,
    mode='predicting'
)

model_2 = VSKNN_STAN(k=500,
                     sample_size=1000,
                     similarity='cosine',
                    #    stan:  
                     lambda_spw=7.24,  
                     lambda_snh=100,  
                     lambda_inh=3.62,  
                    #    vsknn:  
                     lambda_ipw=3.62, 
                     lambda_idf=1
                       )

# Initialize the hybrid model
model = StrategicHybrid(algorithms=[model_1, model_2],
                        thresholds=[15]) # after 15 recommendations, the second algorithm is used
# Fit the hybrid model
model.fit(train)

Initializing offline training agent
Loading pretrained model from: trained agents/retailrocket/20240627_142646 - n_hist 15 state_size 15 action_size 466865.pth
Pretrained model successfully loaded


In [9]:
metrics = [MRR(20),
           HitRate(20)]

evaluate_sessions(pr=model,
                  metrics= metrics,
                  test_data=test,
                  train_data=train,
                  cut_off=20)

START evaluation of  3936  actions in  1125  sessions
    eval process:  0  of  3936  actions:  0.0  %
    eval process:  1000  of  3936  actions:  25.406504065040654  %
    eval process:  2000  of  3936  actions:  50.81300813008131  %
    eval process:  3000  of  3936  actions:  76.21951219512195  %
END evaluation


[('MRR@20: ',
  0.009039712359914503,
  'Bin: ;\nPrecision@20: ;',
  'Pos: ;0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;21;22;23;24;25;26;27;28;29;30;31;32;33;34;35;36;37;38;39;40;41;42;43;44;45;46;47;48;49;50;51;52;53;54;55;56;57;58;59;60;61;62;63;64;65;66;67;68;\nPrecision@20: ;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.26095238095238094;0.19166666666666668;0.1987179487179487;0.16666666666666666;0.061294765840220394;0.029513888888888888;0.15422077922077923;0.046703296703296704;0.12012987012987013;0.21428571428571427;0.07222222222222222;0.0;0.06666666666666667;0.32857142857142857;0.11333333333333333;0.1181818181818182;0.025;0.0;0.016666666666666666;0.041666666666666664;0.09375;0.0;0.037037037037037035;0.0;0.0434173669467787;0.037037037037037035;0.047619047619047616;0.041666666666666664;0.025641025641025644;0.3703703703703704;0.0;0.020833333333333332;0.0;0.1111111111111111;0.018518518518518517;0.3333333333333333;0.16666666666666666;0.16666666666666666;0.0;0.

### 100 interactions
In other words, the model almost only uses the first model to predict next interactions with items.

In [10]:
model_1 = rl_recommender(
    n_history=100,
    file_path=r'trained agents/retailrocket/click=3, addtocart=8, purchase=10/20240627_144343 - n_hist 100 state_size 100 action_size 466865.pth',
    state_size=100,
    action_size=466865,
    mode='predicting'
)

model_2 = VSKNN_STAN(k=500,
                     sample_size=1000,
                     similarity='cosine',
                    #    stan:  
                     lambda_spw=7.24,  
                     lambda_snh=100,  
                     lambda_inh=3.62,  
                    #    vsknn:  
                     lambda_ipw=3.62, 
                     lambda_idf=1
                       )

# Initialize the hybrid model
model = StrategicHybrid(algorithms=[model_1, model_2],
                        thresholds=[100]) # after 100 recommendations, the second algorithm is used
# Fit the hybrid model
model.fit(train)

Initializing offline training agent
Loading pretrained model from: trained agents/retailrocket/20240627_144343 - n_hist 100 state_size 100 action_size 466865.pth
Pretrained model successfully loaded


In [11]:
metrics = [MRR(20),
           HitRate(20)]

evaluate_sessions(pr=model,
                  metrics= metrics,
                  test_data=test,
                  train_data=train,
                  cut_off=20)

START evaluation of  3936  actions in  1125  sessions
    eval process:  0  of  3936  actions:  0.0  %
    eval process:  1000  of  3936  actions:  25.406504065040654  %
    eval process:  2000  of  3936  actions:  50.81300813008131  %
    eval process:  3000  of  3936  actions:  76.21951219512195  %
END evaluation


[('MRR@20: ',
  0.0004387525198624451,
  'Bin: ;\nPrecision@20: ;',
  'Pos: ;0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;21;22;23;24;25;26;27;28;29;30;31;32;33;34;35;36;37;38;39;40;41;42;43;44;45;46;47;48;49;50;51;52;53;54;55;56;57;58;59;60;61;62;63;64;65;66;67;68;\nPrecision@20: ;0.0010370370370370368;0.0;0.0;0.0;0.0;0.0;0.00101010101010101;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;'),
 ('HitRate@20: ',
  0.001422981145499822,
  'Bin: ;\nHitRate@20: ;',
  'Pos: ;0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;21;22;23;24;25;26;27;28;29;30;31;32;33;34;35;36;37;38;39;40;41;42;43;44;45;46;47;48;49;50;51;52;53;54;55;56;57;58;59;60;61;62;63;64;65;66;67;68;\nHitRate@20: ;0.0026666666666666666;0.0;0.0;0.0;0.0;0.0;0.015151515151515152;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.

In [12]:
metrics = [MRR(20),
           HitRate(20)]

evaluate_sessions(pr=model,
                  metrics= metrics,
                  test_data=test,
                  train_data=train,
                  cut_off=20)

START evaluation of  3936  actions in  1125  sessions
    eval process:  0  of  3936  actions:  0.0  %
    eval process:  1000  of  3936  actions:  25.406504065040654  %
    eval process:  2000  of  3936  actions:  50.81300813008131  %
    eval process:  3000  of  3936  actions:  76.21951219512195  %
END evaluation


[('MRR@20: ',
  0.0004387525198624451,
  'Bin: ;\nPrecision@20: ;',
  'Pos: ;0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;21;22;23;24;25;26;27;28;29;30;31;32;33;34;35;36;37;38;39;40;41;42;43;44;45;46;47;48;49;50;51;52;53;54;55;56;57;58;59;60;61;62;63;64;65;66;67;68;\nPrecision@20: ;0.0010370370370370368;0.0;0.0;0.0;0.0;0.0;0.00101010101010101;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;'),
 ('HitRate@20: ',
  0.001422981145499822,
  'Bin: ;\nHitRate@20: ;',
  'Pos: ;0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;21;22;23;24;25;26;27;28;29;30;31;32;33;34;35;36;37;38;39;40;41;42;43;44;45;46;47;48;49;50;51;52;53;54;55;56;57;58;59;60;61;62;63;64;65;66;67;68;\nHitRate@20: ;0.0026666666666666666;0.0;0.0;0.0;0.0;0.0;0.015151515151515152;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.

## Hybrid model (reward: click=0, addtocart=8, purchase=10)

### 5 interactions

In [13]:
model_1 = rl_recommender(
    n_history=5,
    file_path=r'trained agents/retailrocket/click=0, addtocart=8, purchase=10/20240627_151814 - n_hist 5 state_size 5 action_size 466865.pth',
    state_size=5,
    action_size=466865,
    mode='predicting'
)

model_2 = VSKNN_STAN(k=500,
                     sample_size=1000,
                     similarity='cosine',
                    #    stan:  
                     lambda_spw=7.24,  
                     lambda_snh=100,  
                     lambda_inh=3.62,  
                    #    vsknn:  
                     lambda_ipw=3.62, 
                     lambda_idf=1
                       )

# Initialize the hybrid model
model = StrategicHybrid(algorithms=[model_1, model_2],
                        thresholds=[5]) # after 5 recommendations, the second algorithm is used
# Fit the hybrid model
model.fit(train)

Initializing offline training agent
Loading pretrained model from: trained agents/retailrocket/20240627_151814 - n_hist 5 state_size 5 action_size 466865.pth
Pretrained model successfully loaded


### 10 interactions

In [14]:
model_1 = rl_recommender(
    n_history=10,
    file_path=r'trained agents/retailrocket/click=0, addtocart=8, purchase=10/20240627_155644 - n_hist 10 state_size 10 action_size 466865.pth',
    state_size=10,
    action_size=466865,
    mode='predicting'
)

model_2 = VSKNN_STAN(k=500,
                     sample_size=1000,
                     similarity='cosine',
                    #    stan:  
                     lambda_spw=7.24,  
                     lambda_snh=100,  
                     lambda_inh=3.62,  
                    #    vsknn:  
                     lambda_ipw=3.62, 
                     lambda_idf=1
                       )

# Initialize the hybrid model
model = StrategicHybrid(algorithms=[model_1, model_2],
                        thresholds=[10]) # after 10 recommendations, the second algorithm is used
# Fit the hybrid model
model.fit(train)

Initializing offline training agent
Loading pretrained model from: trained agents/retailrocket/20240627_155644 - n_hist 10 state_size 10 action_size 466865.pth
Pretrained model successfully loaded


In [15]:
metrics = [MRR(20),
           HitRate(20)]

evaluate_sessions(pr=model,
                  metrics= metrics,
                  test_data=test,
                  train_data=train,
                  cut_off=20)

START evaluation of  3936  actions in  1125  sessions
    eval process:  0  of  3936  actions:  0.0  %
    eval process:  1000  of  3936  actions:  25.406504065040654  %
    eval process:  2000  of  3936  actions:  50.81300813008131  %
    eval process:  3000  of  3936  actions:  76.21951219512195  %
END evaluation


[('MRR@20: ',
  0.020275660900671243,
  'Bin: ;\nPrecision@20: ;',
  'Pos: ;0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;21;22;23;24;25;26;27;28;29;30;31;32;33;34;35;36;37;38;39;40;41;42;43;44;45;46;47;48;49;50;51;52;53;54;55;56;57;58;59;60;61;62;63;64;65;66;67;68;\nPrecision@20: ;0.00012698412698412698;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.245109201575367;0.2618351886209029;0.28474747474747475;0.2072916666666667;0.23703703703703705;0.26095238095238094;0.19166666666666668;0.1987179487179487;0.16666666666666666;0.061294765840220394;0.029513888888888888;0.15422077922077923;0.046703296703296704;0.12012987012987013;0.21428571428571427;0.07222222222222222;0.0;0.06666666666666667;0.32857142857142857;0.11333333333333333;0.1181818181818182;0.025;0.0;0.016666666666666666;0.041666666666666664;0.09375;0.0;0.037037037037037035;0.0;0.0434173669467787;0.037037037037037035;0.047619047619047616;0.041666666666666664;0.025641025641025644;0.3703703703703704;0.0;0.020833333333333332;0.0;0.11111111

### 15 interactions

In [16]:
model_1 = rl_recommender(
    n_history=15,
    file_path=r'trained agents/retailrocket/click=0, addtocart=8, purchase=10/20240627_153459 - n_hist 15 state_size 15 action_size 466865.pth',
    state_size=15,
    action_size=466865,
    mode='predicting'
)

model_2 = VSKNN_STAN(k=500,
                     sample_size=1000,
                     similarity='cosine',
                    #    stan:  
                     lambda_spw=7.24,  
                     lambda_snh=100,  
                     lambda_inh=3.62,  
                    #    vsknn:  
                     lambda_ipw=3.62, 
                     lambda_idf=1
                       )

# Initialize the hybrid model
model = StrategicHybrid(algorithms=[model_1, model_2],
                        thresholds=[15]) # after 15 recommendations, the second algorithm is used
# Fit the hybrid model
model.fit(train)

Initializing offline training agent
Loading pretrained model from: trained agents/retailrocket/20240627_153459 - n_hist 15 state_size 15 action_size 466865.pth
Pretrained model successfully loaded


In [17]:
metrics = [MRR(20),
           HitRate(20)]

evaluate_sessions(pr=model,
                  metrics= metrics,
                  test_data=test,
                  train_data=train,
                  cut_off=20)

START evaluation of  3936  actions in  1125  sessions
    eval process:  0  of  3936  actions:  0.0  %
    eval process:  1000  of  3936  actions:  25.406504065040654  %
    eval process:  2000  of  3936  actions:  50.81300813008131  %
    eval process:  3000  of  3936  actions:  76.21951219512195  %
END evaluation


[('MRR@20: ',
  0.009039712359914503,
  'Bin: ;\nPrecision@20: ;',
  'Pos: ;0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;21;22;23;24;25;26;27;28;29;30;31;32;33;34;35;36;37;38;39;40;41;42;43;44;45;46;47;48;49;50;51;52;53;54;55;56;57;58;59;60;61;62;63;64;65;66;67;68;\nPrecision@20: ;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.26095238095238094;0.19166666666666668;0.1987179487179487;0.16666666666666666;0.061294765840220394;0.029513888888888888;0.15422077922077923;0.046703296703296704;0.12012987012987013;0.21428571428571427;0.07222222222222222;0.0;0.06666666666666667;0.32857142857142857;0.11333333333333333;0.1181818181818182;0.025;0.0;0.016666666666666666;0.041666666666666664;0.09375;0.0;0.037037037037037035;0.0;0.0434173669467787;0.037037037037037035;0.047619047619047616;0.041666666666666664;0.025641025641025644;0.3703703703703704;0.0;0.020833333333333332;0.0;0.1111111111111111;0.018518518518518517;0.3333333333333333;0.16666666666666666;0.16666666666666666;0.0;0.

### 100 interactions
In other words, the model almost only uses the first model to predict next interactions with items.

In [18]:
model_1 = rl_recommender(
    n_history=100,
    file_path=r'trained agents/retailrocket/click=0, addtocart=8, purchase=10/20240627_150111 - n_hist 100 state_size 100 action_size 466865.pth',
    state_size=100,
    action_size=466865,
    mode='predicting'
)

model_2 = VSKNN_STAN(k=500,
                     sample_size=1000,
                     similarity='cosine',
                    #    stan:  
                     lambda_spw=7.24,  
                     lambda_snh=100,  
                     lambda_inh=3.62,  
                    #    vsknn:  
                     lambda_ipw=3.62, 
                     lambda_idf=1
                       )

# Initialize the hybrid model
model = StrategicHybrid(algorithms=[model_1, model_2],
                        thresholds=[100]) # after 100 recommendations, the second algorithm is used
# Fit the hybrid model
model.fit(train)

Initializing offline training agent
Loading pretrained model from: trained agents/retailrocket/20240627_150111 - n_hist 100 state_size 100 action_size 466865.pth
Pretrained model successfully loaded


In [19]:
metrics = [MRR(20),
           HitRate(20)]

evaluate_sessions(pr=model,
                  metrics= metrics,
                  test_data=test,
                  train_data=train,
                  cut_off=20)

START evaluation of  3936  actions in  1125  sessions
    eval process:  0  of  3936  actions:  0.0  %
    eval process:  1000  of  3936  actions:  25.406504065040654  %
    eval process:  2000  of  3936  actions:  50.81300813008131  %
    eval process:  3000  of  3936  actions:  76.21951219512195  %
END evaluation


[('MRR@20: ',
  0.0,
  'Bin: ;\nPrecision@20: ;',
  'Pos: ;0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;21;22;23;24;25;26;27;28;29;30;31;32;33;34;35;36;37;38;39;40;41;42;43;44;45;46;47;48;49;50;51;52;53;54;55;56;57;58;59;60;61;62;63;64;65;66;67;68;\nPrecision@20: ;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;'),
 ('HitRate@20: ',
  0.0,
  'Bin: ;\nHitRate@20: ;',
  'Pos: ;0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;21;22;23;24;25;26;27;28;29;30;31;32;33;34;35;36;37;38;39;40;41;42;43;44;45;46;47;48;49;50;51;52;53;54;55;56;57;58;59;60;61;62;63;64;65;66;67;68;\nHitRate@20: ;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.